<a href="https://colab.research.google.com/github/kshitij9969/Indicator_of_sucess_dataset/blob/main/Indicator_of_sucess_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
# To do: 
# Two types of data needs to be extracted
# 1. Indexes
# 2. True values

# 1. Cost of living index by city
# 2. Cost of living index by country
# 3. Property price index
# 4. Quality of life index
# 5. Crime index
# 6. Health care index
# 7. Pollution index
# 8. Traffic index
# 9. 


# 3. Cost of living estimator by city
# 4. Food prices by city
# 5. Food prices by country
# 6. Prices of other commodity by city and country
# 7. Basket of prices by city
# 8. Taxi fare by city
# 9. Gas prices by city


# Cost of living index

### Cost of living index over the years. 

In [30]:
response = requests.get("https://www.numbeo.com/cost-of-living/rankings.jsp")

In [3]:
df = pd.read_html(response.text)

In [4]:
df

[    0                                                  1
 0 NaN  whenDocReady(function() {  jQuery('#city_selec...,
      Rank  ... Local Purchasing Power Index
 0     NaN  ...                        70.34
 1     NaN  ...                        90.79
 2     NaN  ...                       111.12
 3     NaN  ...                        97.09
 4     NaN  ...                       108.91
 ..    ...  ...                          ...
 558   NaN  ...                        43.05
 559   NaN  ...                        20.94
 560   NaN  ...                        23.23
 561   NaN  ...                        65.39
 562   NaN  ...                        35.16
 
 [563 rows x 8 columns]]

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
table = soup.find("table", attrs={"id": "t2"})

In [7]:
df = pd.read_html(str(table))[0]

In [8]:
# Cleaning columns

for col in df.columns:
    df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)


In [9]:
# Split city and country

df['Country'] = df['City'].apply(lambda x: x.split(", ")[-1].strip())

In [10]:
df['City'] = df['City'].apply(lambda x: x.split(", ")[0].strip())

In [11]:
df.reset_index(inplace=True)

In [12]:
df['Rank'] = df['index']

In [13]:
df.drop(columns=['index'], inplace=True)

In [14]:
df['Rank'] = df['Rank'].apply(lambda x: x + 1)

In [15]:
df['Year'] = 2021
df['Month'] = 6
df['Day'] = 1

In [16]:
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

In [48]:
from requests.api import request
# Now collect for all other years and months

# Country links: 
links = [
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=002", "Africa"), # Africa
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=019", "America"), # America
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=142", "Asia"), # Asia
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=150", "Europe"), # Europe
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=009", "Oceania") # Oceania
]

# Collecting links for regions

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date'])

years_list = ['2021-mid', '2021', '2020-mid', '2020', '2019-mid', '2019', '2018-mid', '2018', '2017-mid', '2017', '2016-mid', '2016', '2015-mid', '2015', '2014-mid', '2014', '2013', '2012', '2011', '2010', '2009']

for (i, continent) in links:
    res = requests.get(i)
    soup = BeautifulSoup(res.text)
    try:
        spans = soup.find("div", attrs={"class": "select_region_links"}).find_all("span", attrs={"class": "nobreak"})
    except Exception as e:
        # Oceania doesn't have any regions
        print(e)
        print(i)
    print(spans)
    for span in spans:
        url = span.find("a", href=True)['href']
        resp = requests.get(url)
        soup1 = BeautifulSoup(resp.text)
        # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
        # print(resp.text)
        table = soup.find("table", attrs={"id": "t2"})
        df = pd.read_html(str(table))[0]
        # Clean columns
        for col in df.columns:
            df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
        
        # Correct Rank
        df.reset_index(inplace=True)
        df['Rank'] = df['index']
        df['Rank'] = df['Rank'].apply(lambda x: x + 1)
        df.drop(columns=['index'], inplace=True)
        # Add month year and day
        df['Year'] = 2021
        df['Month'] = 1
        df['Day'] = 1

        df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
        # Add country and continent
        df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
        df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
        df['Continent'] = continent
        df['Region'] = soup1.find("h1").text.split(":")[0].strip()
        print(url)
        print(years_list)
    break
        # print(url)
        # print(years_list)
        # region_name = soup1.find("h1").text.split(":")[0].strip()
        # print(region_name)






[<span class="nobreak">
<a class="ui-state-default region_link" href="https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&amp;region=014">Eastern Africa</a>
</span>, <span class="nobreak">
<a class="ui-state-default region_link" href="https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&amp;region=017">Middle Africa</a>
</span>, <span class="nobreak">
<a class="ui-state-default region_link" href="https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&amp;region=015">Northern Africa</a>
</span>, <span class="nobreak">
<a class="ui-state-default region_link" href="https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&amp;region=018">Southern Africa</a>
</span>, <span class="nobreak">
<a class="ui-state-default region_link" href="https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&amp;region=011">Western Africa</a>
</span>]
https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=014
['2021-m

In [44]:
from requests.api import request
# Now collect for all other years and months

# Country links: 
links = [
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=002", "Africa"), # Africa
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=019", "America"), # America
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=142", "Asia"), # Asia
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=150", "Europe"), # Europe
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=009", "Oceania") # Oceania
]

# Collecting links for regions

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date'])

years_list = ['2021-mid', '2021', '2020-mid', '2020', '2019-mid', '2019', '2018-mid', '2018', '2017-mid', '2017', '2016-mid', '2016', '2015-mid', '2015', '2014-mid', '2014', '2013', '2012', '2011', '2010', '2009']

for (i, continent) in links:
    res = requests.get(i)
    soup = BeautifulSoup(res.text)
    try:
        spans = soup.find("div", attrs={"class": "select_region_links"}).find_all("span", attrs={"class": "nobreak"})
    except Exception as e:
        # Oceania doesn't have any regions
        print(e)
        print(i)
    print(spans)
    for span in spans:
        url = span.find("a", href=True)['href']
        resp = requests.get(url)
        soup1 = BeautifulSoup(resp.text)
        # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
        # print(resp.text)
        table = soup.find("table", attrs={"id": "t2"})
        df = pd.read_html(str(table))[0]
        # Clean columns
        for col in df.columns:
            df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
        
        # Correct Rank
        df.reset_index(inplace=True)
        df['Rank'] = df['index']
        df['Rank'] = df['Rank'].apply(lambda x: x + 1)
        df.drop(columns=['index'], inplace=True)
        # Add month year and day
        df['Year'] = 2021
        df['Month'] = 1
        df['Day'] = 1

        df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
        # Add country and continent
        df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
        df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
        df['Continent'] = continent
        df['Region'] = soup1.find("h1").text.split(":")[0].strip()
        print(url)
        print(years_list)
    break
        # print(url)
        # print(years_list)
        # region_name = soup1.find("h1").text.split(":")[0].strip()
        # print(region_name)






,Rank,City,Cost_of_Living_Index,Rent_Index,Cost_of_Living_Plus_Rent_Index,Groceries_Index,Restaurant_Price_Index,Local_Purchasing_Power_Index,Year,Month,Day,Date,Country,Continent,Region
0,1,Addis Ababa,58.92,15.40,38.13,38.54,20.60,10.29,2021,1,1,2021-01-01,Ethiopia,Africa,Eastern Africa
1,2,Abidjan,55.73,24.40,40.76,47.72,34.57,7.03,2021,1,1,2021-01-01,Ivory Coast,Africa,Eastern Africa
2,3,Harare,52.33,10.13,32.17,43.21,43.23,19.78,2021,1,1,2021-01-01,Zimbabwe,Africa,Eastern Africa
3,4,Johannesburg,46.00,15.82,31.58,35.81,42.29,71.19,2021,1,1,2021-01-01,South Africa,Africa,Eastern Africa
4,5,Pretoria,44.70,12.65,29.39,35.04,36.90,75.29,2021,1,1,2021-01-01,South Africa,Africa,Eastern Africa
5,6,Gaborone,42.84,11.62,27.92,35.82,48.29,60.12,2021,1,1,2021-01-01,Botswana,Africa,Eastern Africa
6,7,Cape Town,42.24,23.43,33.26,36.26,37.40,66.96,2021,1,1,2021-01-01,South Africa,Africa,Eastern Africa
7,8,Accra,42.18,21.24,32.18,43.66,31.65,17.58,2021,1,1,2021-01-01,Ghana,Africa,Eastern Africa
8,9,Marrakech,41.73,10.03,26.58,33.27,38.38,30.35,2021,1,1,2021-01-01,Morocco,Africa,Eastern Africa
9,10,Windhoek,41.08,16.25,29.22,38.90,34.06,58.94,2021,1,1,2021-01-01,Namibia,Africa,Eastern Africa
